In [1]:
import geopandas as gpd
import requests
import json
import geopandas as gpd
import networkx as nx
from shapely.geometry import mapping
import csv
from tqdm import tqdm

In [2]:
# Cargar shapefile local
shapefile_path = "C:/Users/Michael/Documents/ml_zc/ecu_adm_2024/ecu_adm_adm3_2024.shp"
gdf = gpd.read_file(shapefile_path)

In [3]:
parroquias = gdf[['ADM3_PCODE', 'geometry']]
# Solo selecciono dos filas, para reducir la carga de procesamiento

In [4]:
# parroquias = parroquias[parroquias['ADM3_PCODE'].isin(['EC050651', 'EC010951'], )]
# Este polígono no no tiene info en las otras fuentes
parroquias = parroquias[parroquias['ADM3_PCODE'] != 'EC160156']

In [5]:
#parroquias[parroquias['ADM3_PCODE'] == 'EC040554']
parroquias = parroquias[163:]

In [ ]:
# Fechas del 31 de diciembre de 2014 a 2024
fechas = [f"{a}-12-31" for a in range(2014, 2025)]
años = [str(a) for a in range(2014, 2025)]

# Endpoint de Ohsome
url = "https://api.ohsome.org/v1/elements/geometry"

# Ruta a tu archivo
csv_output_path = 'C:/Users/Michael/Documents/ml_zc/datosObtenidos/analisis_redes.csv'

# Crear archivo y escribir encabezado
with open(csv_output_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['ADM3_PCODE', 'Año', 'densidad_agua', 'numero_componentes_agua'])
    writer.writeheader()

    for num_fila in tqdm(range(parroquias.shape[0]), desc="Procesando parroquias"):
        
        parroquia = parroquias.iloc[num_fila]
        name_parr = parroquia['ADM3_PCODE']
        print(f"Procesando: {name_parr}")

        # Construir el GeoJSON de la parroquia
        geojson = {
            "type": "FeatureCollection",
            "features": [
                {
                    "type": "Feature",
                    "properties": {"id": name_parr},
                    "geometry": mapping(parroquia['geometry'])
                }
            ]
        }

        # Parámetros para la consulta (eliminamos el parámetro 'format')
        params = {
            "bpolys": json.dumps(geojson),
            "time": ",".join(fechas),
            "filter": "highway=*"
        }

        
        try:
            res = requests.post(url, data=params, json=geojson)

            if res.status_code == 200:
                data = res.json()

                # Convertir los features de la respuesta a un GeoDataFrame
                gdf = gpd.GeoDataFrame.from_features(data["features"])

                # Asegúrate de que tiene el CRS adecuado
                gdf = gdf.set_crs("EPSG:4326", allow_override=True)

                # Verifica si los datos se cargaron correctamente
                # print(gdf.head())
                
                # Filtrar solo los LineString
                if 'LineString' in gdf.geometry.type.unique():
                    gdf = gdf[gdf.geometry.type == "LineString"]

                    # Crear un grafo para un año específico
                    for año in años:

                        print(f'  {año}:')

                        gdf_año = gdf[gdf["@snapshotTimestamp"].str.startswith(año)]

                        # Crear grafo dirigido o no dirigido
                        G = nx.Graph()

                        for line in gdf_año.geometry:
                            coords = list(line.coords)
                            for i in range(len(coords) - 1):
                                start = coords[i]
                                end = coords[i + 1]
                                G.add_edge(start, end)

                        # > > DENSIDAD DE RED

                        # Calcular la densidad de la red
                        num_nodes = len(G.nodes)
                        num_edges = len(G.edges)

                        # Densidad (usamos fórmula para grafos no dirigidos)
                        print(f'    Nodos: {num_nodes}')

                        if num_nodes > 1: 
                            density = 2 * num_edges / (num_nodes * (num_nodes - 1))
                        else:
                            density = 0

                        print(f'    Densidad de la red: {density}')

                        # >> COMPONENTES CONEXAS
                        components = list(nx.connected_components(G.to_undirected()))  # Para asegurar que la red es no dirigida
                        num_components = len(components)
                        print(f'    Número de componentes conexas: {num_components}')

                        dic = {'ADM3_PCODE': name_parr, 'Año': año, 'densidad': density, 'numero_componentes': num_components}
                        writer.writerow(dic)
                
                else:
                    print(f"No hubo líenea.")

            else:
                print(f"❌ Error {res.status_code} para {name_parr}: {res.text}")

        except Exception as e:
            print(f"⚠️ Error al procesar {name_parr}: {e}")

Procesando parroquias:   0%|                                                                   | 0/878 [00:00<?, ?it/s]

Procesando: EC040554
  2014:
    Nodos: 371
    Densidad de la red: 0.005827930356232243
    Número de componentes conexas: 2
  2015:
    Nodos: 377
    Densidad de la red: 0.005742423387324341
    Número de componentes conexas: 1
  2016:
    Nodos: 566
    Densidad de la red: 0.0037274461365270958
    Número de componentes conexas: 1
  2017:
    Nodos: 835
    Densidad de la red: 0.002487112106721808
    Número de componentes conexas: 1
  2018:
    Nodos: 1053
    Densidad de la red: 0.0019589151401572186
    Número de componentes conexas: 1
  2019:
    Nodos: 1321
    Densidad de la red: 0.001554148601839745
    Número de componentes conexas: 1
  2020:
    Nodos: 2069
    Densidad de la red: 0.0009872175889267094
    Número de componentes conexas: 2
  2021:
    Nodos: 3715
    Densidad de la red: 0.0005485047664397416
    Número de componentes conexas: 2
  2022:
    Nodos: 3762
    Densidad de la red: 0.0005416682392290783
    Número de componentes conexas: 2
  2023:
    Nodos: 3956


Procesando parroquias:   0%|                                                         | 1/878 [00:11<2:51:33, 11.74s/it]

    Número de componentes conexas: 2
  2024:
    Nodos: 3956
    Densidad de la red: 0.0005146369866253184
    Número de componentes conexas: 2
Procesando: EC040555


Procesando parroquias:   0%|▏                                                        | 2/878 [00:19<2:20:23,  9.62s/it]

...

  2014:
    Nodos: 97
    Densidad de la red: 0.020403780068728523
    Número de componentes conexas: 2
  2015:
    Nodos: 501
    Densidad de la red: 0.00417564870259481
    Número de componentes conexas: 4
  2016:
    Nodos: 668
    Densidad de la red: 0.0031825404662937993
    Número de componentes conexas: 3
  2017:
    Nodos: 668
    Densidad de la red: 0.0031825404662937993
    Número de componentes conexas: 3
  2018:
    Nodos: 736
    Densidad de la red: 0.0028800650695060635
    Número de componentes conexas: 3
  2019:
    Nodos: 736
    Densidad de la red: 0.0028800650695060635
    Número de componentes conexas: 3
  2020:
    Nodos: 831
    Densidad de la red: 0.0025372247111188437
    Número de componentes conexas: 4
  2021:
    Nodos: 965
    Densidad de la red: 0.002169286006062821
    Número de componentes conexas: 4
  2022:
    Nodos: 965
    Densidad de la red: 0.002169286006062821
    Número de componentes conexas: 4
  2023:
    Nodos: 1227
    Densidad de la red: 0.00

Procesando parroquias:   1%|▎                                                        | 5/878 [00:57<3:09:35, 13.03s/it]

    Número de componentes conexas: 52
Procesando: EC050151
  2014:
    Nodos: 1493
    Densidad de la red: 0.0013674179235000153
    Número de componentes conexas: 11
  2015:
    Nodos: 2020
    Densidad de la red: 0.0010067722968433545
    Número de componentes conexas: 16
  2016:
    Nodos: 2793
    Densidad de la red: 0.0007237701293758341
    Número de componentes conexas: 37
  2017:
    Nodos: 2891
    Densidad de la red: 0.0007016166386794
    Número de componentes conexas: 34
  2018:
    Nodos: 2893
    Densidad de la red: 0.0007011248116907363
    Número de componentes conexas: 34
  2019:
    Nodos: 3172
    Densidad de la red: 0.0006388682428200396
    Número de componentes conexas: 34
  2020:
    Nodos: 3172
    Densidad de la red: 0.0006390670813643346
    Número de componentes conexas: 34
  2021:
    Nodos: 4540
    Densidad de la red: 0.0004459636648799004
    Número de componentes conexas: 37
  2022:
    Nodos: 4460
    Densidad de la red: 0.00045386113840401385
    Númer

Procesando parroquias:   1%|▍                                                        | 6/878 [01:08<2:55:21, 12.07s/it]

    Nodos: 6517
    Densidad de la red: 0.0003097155449227421
    Número de componentes conexas: 42
Procesando: EC050152
